In [1]:
#auto reload 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os 
from datasets import load_dataset


c:\Users\Alhus\.conda\envs\mlaproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
wmt14 = load_dataset("wmt14", "fr-en", data_dir="data/")

# Accessing example data
train_data = wmt14["train"]
val_data = wmt14["validation"]


c:\Users\Alhus\.conda\envs\mlaproject\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [21]:
examples = 4 
#check some data and their translation
for i, t in zip(range(examples), train_data):
    ex = t["translation"]
    print(ex["fr"], " -> ", ex["en"], "\n\n")

Reprise de la session  ->  Resumption of the session 


Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.  ->  I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period. 


Comme vous avez pu le constater, le grand "bogue de l'an 2000" ne s'est pas produit. En revanche, les citoyens d'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles.  ->  Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful. 


Vous avez souhaité un débat à ce sujet dans les prochains jours, au cours de cette période de session.  ->  You 

In [23]:
#tokenization using MosesTokenizer
from mosestokenizer import MosesTokenizer, MosesDetokenizer
mt = MosesTokenizer('en')
md = MosesDetokenizer('en')


stdbuf was not found; communication with perl may hang due to stdio buffering.


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [25]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.moses import MosesDetokenizer

# Sample sentence
sentence = "This is an example sentence with words that may not be in the shortlist."

# Your shortlist of the 30,000 most used words
shortlist = set(["this", "is", "an", "example", "sentence", "with", "words", "that", "may", "not", "be", "in", "the", "shortlist"])

# Tokenize using NLTK's word_tokenize
tokens = word_tokenize(sentence)

# Replace unknown words with "UNK"
tokens = ['UNK' if token.lower() not in shortlist else token for token in tokens]

# Detokenize using NLTK's MosesDetokenizer
detokenizer = MosesDetokenizer()
detokenized_sentence = detokenizer.detokenize(tokens, return_str=True)

# Print the result
print(detokenized_sentence)


ModuleNotFoundError: No module named 'nltk.tokenize.moses'